In [ ]:
import torch
torch.__version__
# import tensorrt as trt

In [ ]:
import mmpose 
mmpose.__version__

In [ ]:
import pandas as pd
file=pd.read_pickle('Data/pickle_file/Class_Error1.pkl')
file2=pd.read_pickle('Data/pickle_file/Class_Error2.pkl')
file3=pd.read_pickle('Data/pickle_file/Class_Error3.pkl')

In [ ]:
len(file), len(file2), len(file3)

In [ ]:
import cv2 
import os.path as osp
import os
def extract_frame(video_path):
    dname = 'temp'
    os.makedirs(dname, exist_ok=True)
    frame_tmpl = osp.join(dname, 'img_{:05d}.jpg')
    cap = cv2.VideoCapture(video_path)
    frame_paths = []
    cnt = 0
    while(cap.isOpened()):
        flag, frame = cap.read()
        if flag:
            frame_path = frame_tmpl.format(cnt + 1)
            frame_paths.append(frame_path)

            cv2.imwrite(frame_path, frame)
            cnt += 1
            
    return frame_paths


In [ ]:
# fp=extract_frame('C:/Users/ADMIN/Videos/Desktop/Desktop 2023.02.10 - 21.10.59.01.mp4')

In [ ]:
from mmdet.apis import inference_detector, init_detector
from mmpose.apis import inference_top_down_pose_model, init_pose_model,process_mmdet_results,vis_pose_result
import mmcv
import numpy as np
import shutil
# pose_config = 'configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/coco/hrnet_w48_coco_256x192.py'
# pose_checkpoint = 'https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth'
# det_config = 'demo/mmdetection_cfg/faster_rcnn_r50_fpn_coco.py'
# det_checkpoint = 'https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'
#############################
# pose_config = 'Pose/hrnet_w48_coco_256x192.py'
# pose_checkpoint = 'Pose/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth'
###############
pose_config = 'mmpose/configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/coco/hrnet_w48_coco_256x192.py'
pose_checkpoint = 'Pose/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth'
###########
# det_config = 'Pose/yolox_s_8x8_300e_coco.py'
# det_checkpoint = 'Pose/yolox_s_8x8_300e_coco_20211121_095711-4592a793.pth'
##############
det_config = 'mmdetection/configs/yolox/yolox_s_8x8_300e_coco.py'
det_checkpoint = 'Pose/yolox_s_8x8_300e_coco_20211121_095711-4592a793.pth'

# initialize pose model
pose_model = init_pose_model(pose_config, pose_checkpoint)
# initialize detector
det_model = init_detector(det_config, det_checkpoint)
img = 'mmpose/tests/data/coco/000000196141.jpg'

In [ ]:
img = 'temp/img_00050.jpg'
mmdet_results = inference_detector(det_model, img)
person_results = process_mmdet_results(mmdet_results, cat_id=1)
pose_results= inference_top_down_pose_model(
    pose_model,
    img,
    person_results,
    bbox_thr=0.8,
    format='xyxy',
    dataset=pose_model.cfg.data.test.type,
    )[0]

In [ ]:
mmdet_results = inference_detector(det_model, img)
a=mmdet_results[0]
# extract person (COCO_ID=1) bounding boxes from the detection results
person_results = process_mmdet_results(mmdet_results, cat_id=1)

# inference pose
anno=[]
pose_results= inference_top_down_pose_model(
    pose_model,
    img,
    person_results,
    bbox_thr=0.8,
    format='xyxy',
    dataset=pose_model.cfg.data.test.type,
    )[0]

# show pose estimation results
# vis_result = vis_pose_result(
#     pose_model,
#     img,
#     pose_results,
#     dataset=pose_model.cfg.data.test.type,
#     show=False)

In [ ]:
len(person_results)

In [ ]:
anno = dict()
anno['keypoint'] = pose_results[..., :2]

In [ ]:
# bbox_sorted=[]
# index_bbox=[]
# Area_bbox=[]
# for index,person_result in enumerate(pose_results):
#     if person_result['bbox'][-1]>0.5: 
#         bbox_sorted.append(dict(bbox=person_result['bbox']))
#         index_bbox.append(index)
#         Width=person_result['bbox'][2]-person_result['bbox'][0]
#         Height=person_result['bbox'][3]-person_result['bbox'][1]
#         Area_bbox.append(Width*Height)

In [ ]:
filterlist=[]
for person in pose_results:
    W = person['bbox'][2] - person['bbox'][0]
    H = person['bbox'][3] - person['bbox'][1]
    conf  = person['bbox'][4]
    Area = H*W
    filterlist.append(Area)
tensorfilter=torch.tensor([filterlist])

In [ ]:
# _,indexmax=torch.max(torch.tensor([filterlist]),dim=1)
# indexmax
newlist=[pose_results[0],pose_results[1]]

In [ ]:
pose_results

In [ ]:
vis_result = vis_pose_result(
    pose_model,
    img,
    pose_results,
    dataset=pose_model.cfg.data.test.type,
    show=False)

In [ ]:
len(pose_results)

In [ ]:
# vis_result = cv2.resize(vis_result, dsize=None, fx=0.5, fy=0.5)
# # cv2.imshow(vis_result)

In [ ]:
import matplotlib.pyplot  as plt
plt.imshow(vis_result)

In [ ]:
def detection_inference(det_config, det_checkpoint ,frame_paths, det_score_thr=0.5,device='cuda' ):
    model = init_detector(det_config, det_checkpoint, device)
    assert model.CLASSES[0] == 'person', ('We require you to use a detector '
                                          'trained on COCO')
    results = []
    print('Performing Human Detection for each frame')
    prog_bar = mmcv.ProgressBar(len(frame_paths))
    for frame_path in frame_paths:
        result = inference_detector(model, frame_path)
        # We only keep human detections with score larger than det_score_thr
        result = result[0][result[0][:, 4] >= det_score_thr]
        results.append(result)
        prog_bar.update()
    return results

def pose_inference(pose_config,pose_checkpoint, frame_paths, det_results, device='cuda'):
    model = init_pose_model(pose_config, pose_checkpoint, device)
    print('Performing Human Pose Estimation for each frame')
    prog_bar = mmcv.ProgressBar(len(frame_paths))

    num_frame = len(det_results)
    num_person = max([len(x) for x in det_results])
    kp = np.zeros((num_person, num_frame, 17, 3), dtype=np.float32)

    for i, (f, d) in enumerate(zip(frame_paths, det_results)):
        # Align input format
        d = [dict(bbox=x) for x in list(d) if x[-1] > 0.5]
        pose = inference_top_down_pose_model(model, f, d, format='xyxy')[0]
        for j, item in enumerate(pose):
            kp[j, i] = item['keypoints']
        prog_bar.update()
    return kp


def ntu_pose_extraction(vid,det_config, det_checkpoint,pose_config,pose_checkpoint,label, det_score_thr=0.5,device='cuda'):
    frame_paths = extract_frame(vid)
    det_results = detection_inference(det_config, det_checkpoint ,frame_paths, det_score_thr,device)
    image = cv2.imread(frame_paths[0])
    image_shape = (image.shape[0], image.shape[1])
    pose_results = pose_inference(pose_config,pose_checkpoint, frame_paths, det_results, device)
    anno = dict()
    anno['keypoint'] = pose_results[..., :2]
    anno['keypoint_score'] = pose_results[..., 2]
    anno['frame_dir'] = osp.splitext(osp.basename(vid))[0]
    anno['img_shape'] = image_shape
    anno['original_shape'] = image_shape
    anno['total_frames'] = pose_results.shape[1]
    anno['label'] = label
    shutil.rmtree(osp.dirname(frame_paths[0]))
    return anno

In [1]:
import numpy as np
from mmpose.apis import inference_topdown
from mmpose.apis import init_model as init_pose_estimator
from mmdet.apis import init_detector,inference_detector
import cv2
from mmpose.evaluation.functional import nms
from mmpose.registry import VISUALIZERS
from mmpose.structures import merge_data_samples
import mmengine
from mmengine.registry import init_default_scope
import time

c:\Users\PhuTuan\anaconda3\envs\pythonProject\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
img = 'mmdeploy/demo/resources/human-pose.jpg'
device = 'cuda:0'
det_config = 'mmdetection/configs/yolox/yolox_s_8xb8-300e_coco.py'
pose_config = 'mmpose/configs/body_2d_keypoint/topdown_heatmap/coco/td-hm_hrnet-w48_8xb32-210e_coco-256x192.py'
pose_checkpoint = 'Pose/td-hm_hrnet-w48_8xb32-210e_coco-256x192-0e67c616_20220913.pth'
det_checkpoint = 'Pose/yolox_s_8x8_300e_coco_20211121_095711-4592a793.pth'

In [3]:
det_model = init_detector(det_config,det_checkpoint)
pose_model = init_pose_estimator(pose_config,pose_checkpoint)

Loads checkpoint by local backend from path: Pose/yolox_s_8x8_300e_coco_20211121_095711-4592a793.pth
Loads checkpoint by local backend from path: Pose/td-hm_hrnet-w48_8xb32-210e_coco-256x192-0e67c616_20220913.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: data_preprocessor.mean, data_preprocessor.std



In [4]:
pose_model.cfg.visualizer.radius = 3
pose_model.cfg.visualizer.line_width = 1
visualizer = VISUALIZERS.build(pose_model.cfg.visualizer)
# the dataset_meta is loaded from the checkpoint and
# then pass to the model in init_pose_model
visualizer.set_dataset_meta(pose_model.dataset_meta)

05/20 23:50:12 - mmengine - WARNING - `Visualizer` backend is not initialized because save_dir is None.


In [5]:
def visualize_img(img_path, detector, pose_estimator, visualizer,
                  show_interval, out_file):
    """Visualize predicted keypoints (and heatmaps) of one image."""

    # predict bbox
    init_default_scope(detector.cfg.get('default_scope', 'mmdet'))
    detect_result = inference_detector(detector, img_path)
    pred_instance = detect_result.pred_instances.cpu().numpy()
    bboxes = np.concatenate(
        (pred_instance.bboxes, pred_instance.scores[:, None]), axis=1)
    bboxes = bboxes[np.logical_and(pred_instance.labels == 0,
                                   pred_instance.scores > 0.3)]
    bboxes = bboxes[nms(bboxes, 0.3)][:, :4]

    # predict keypoints
    pose_results = inference_topdown(pose_estimator, img_path, bboxes)
    data_samples = merge_data_samples(pose_results)

    # show the results
    img = cv2.imread(img_path)

    visualizer.add_datasample(
        'result',
        img,
        data_sample=data_samples,
        draw_gt=False,
        draw_heatmap=True,
        draw_bbox=True,
        show=False,
        wait_time=show_interval,
        out_file=out_file,
        kpt_thr=0.3)

In [6]:
Start = time.time()
visualize_img(
    img,
    det_model,
    pose_model,
    visualizer,
    show_interval=0,
    out_file=None)
vis_result = visualizer.get_image()
TimeRun = time.time()-Start

05/20 23:50:12 - mmengine - WARNING - The current default scope "mmpose" is not "mmdet", `init_default_scope` will force set the currentdefault scope to "mmdet".
05/20 23:50:15 - mmengine - WARNING - The current default scope "mmdet" is not "mmpose", `init_default_scope` will force set the currentdefault scope to "mmpose".


c:\Users\PhuTuan\anaconda3\envs\pythonProject\lib\site-packages\torch\functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [7]:
TimeRun

3.165093183517456

In [ ]:
# init_default_scope(det_model.cfg.get('default_scope', 'mmdet'))
# detect_result = inference_detector(det_model, img)
# pred_instance = detect_result.pred_instances.cpu().numpy()
# bboxes = np.concatenate(
#     (pred_instance.bboxes, pred_instance.scores[:, None]), axis=1)
# bboxes = bboxes[np.logical_and(pred_instance.labels == 0,
#                                 pred_instance.scores > 0.3)]
# bboxes = bboxes[nms(bboxes, 0.3)][:, :4]

# # predict keypoints
# pose_results = inference_topdown(pose_model, img, bboxes)
# data_samples = merge_data_samples(pose_results)
# img_read = cv2.imread(img)
# visualizer.add_datasample(
#         'result',
#         img_read,
#         data_sample=data_samples,
#         draw_gt=False,
#         draw_heatmap=True,
#         draw_bbox=True,
#         show=False,
#         wait_time=0,
#         out_file=None,
#         kpt_thr=0.3)

In [6]:
cv2.imshow('',vis_result)
cv2.waitKey(-1)

NameError: name 'vis_result' is not defined